In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import gensim
import cython
import nltk
import pymystem3
import sys
import os
import itertools

from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from pymystem3 import Mystem

from inscriptis import get_text
import pickle
import progressbar
from datetime import datetime
from multiprocessing.dummy import Pool as ThreadPool

In [2]:
def html2text_bs_visible(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")    
    [s.extract() for s in soup(['style', 'script', '[document]', 'head', 'title'])]
    return soup.get_text()

In [3]:
def html2text_bs(raw_html):
    soup = BeautifulSoup(raw_html, "html.parser")
    [s.extract() for s in soup(['script', 'style'])]
    return soup.get_text()

In [4]:
def html2text_inscryptis(raw_html):
    return get_text(raw_html)

In [5]:
def html2word(raw_html, stopWords, to_text=html2text_bs, tokenizer=nltk.word_tokenize):
    lemmerRus = Mystem() 
    lemmerEng = WordNetLemmatizer() 
    
    words = tokenizer(to_text(raw_html).lower())
    wordsFiltered = list()
    
    for w in words:
        w = ''.join(list(filter(lambda x: x.isalpha() or x.isdigit() or x==" ", w)))
        if w not in stopWords:
            w = lemmerEng.lemmatize(w)
            w = lemmerRus.lemmatize(w)
            
            for ww in w[:-1]:
                if len(ww) > 1:
                    wordsFiltered.append(ww)
    
    return wordsFiltered

In [6]:
stopWords = set(stopwords.words("english")) |  set(stopwords.words("russian"))

In [7]:
queries = pd.read_table("queries_new.txt", header=None, sep="\t")

queries = queries.rename(columns={
    0: "queries_id",
    1: "query"
})

In [8]:
lemmerRus = Mystem() 
lemmerEng = WordNetLemmatizer() 

queries_list = []
for i in queries.index:
    print(str(i) + "/" + str(len(queries)), end="\r")
    words = word_tokenize(queries.iloc[i]['query'])
    wordsFiltered = list()
 
    for w in words:
        
        if len(w) <= 1:
            continue
        
        w = ''.join(list(filter(lambda x: x.isalpha() or x.isdigit() or x==" ", w)))
        if w not in stopWords:
            w = lemmerEng.lemmatize(w)
            w = lemmerRus.lemmatize(w)
            # print(w)
            for ww in w[:-1]:
                if len(ww) > 1:
                    wordsFiltered.append(ww)
    
    query = ""
    for word in wordsFiltered:
        query = query + str(word) + " "
    query = query[:-1]
    
    queries_list.append(query)

queries['query'] = queries_list

In [9]:
queries

,queries_id,query
0,1,03l 198 119
1,2,тысяча доллар переводить рубль днр
2,3,10 достопримечательность пекин
3,4,1000 купон челябинск
4,5,115407 почтовый отделение адрес
5,6,117 ук рф истязание несовершеннолетний
6,7,14 летний модель близняшка доводить анорексия ...
7,8,1530 mskobr
8,9,18210202090071010160 кбк 2019
9,10,солнечный 30а индекс омск


In [10]:
urls = pd.read_table("urls.numerate.txt", header=None)

urls = urls.rename(columns={
    0: "url_id",
    1: "url"
})

In [11]:
# print("Progress {}/{}".format(i, len(os.listdir("content/20190128/"))), end="\r")
def read_url_from_doc(doc):
    f = open("content/20190128/" + str(doc))
    url_doc = [f.readline()[:-1], str(doc)]
    # print("Progress {}".format(i, end="\r"))
    return url_doc

In [12]:
%%time
pool = ThreadPool(4)
result = pool.map(read_url_from_doc, os.listdir("content/20190128/"))
pool.close()
pool.join()

CPU times: user 4.18 s, sys: 3.26 s, total: 7.44 s
Wall time: 9.15 s


In [13]:
url_df = pd.DataFrame(result)
url_df = url_df.rename(columns={
    0: "url",
    1: "file"
})

In [14]:
adj = url_df.set_index("url").join(urls.set_index("url"), how='inner').reset_index()[['file', 'url_id']]

In [15]:
adj.head(10)

,file,url_id
0,doc.68956.dat,39108
1,doc.68524.dat,33814
2,doc.32008.dat,57609
3,doc.05041.dat,46941
4,doc.37756.dat,33563
5,doc.29291.dat,66899
6,doc.09026.dat,66228
7,doc.53978.dat,31241
8,doc.64543.dat,53238
9,doc.17993.dat,50462


In [16]:
doc_index = pd.DataFrame({"data": os.listdir("content/20190128/"),
                         "index": range(len(os.listdir("content/20190128/")))})

In [17]:
links = doc_index.set_index('data').join(adj.set_index('file'), how='inner').reset_index().rename(columns={
    "level_0": "file",
    "url_id": "docId"
})
links['docId'] = links['docId'].astype('int')

In [18]:
links.head(5)

,file,index,docId
0,doc.68956.dat,0,39108
1,doc.68524.dat,1,33814
2,doc.32008.dat,2,57609
3,doc.05041.dat,3,46941
4,doc.37756.dat,4,33563


In [6]:
class MySentences(object):
    def __init__(self):
        pass
    
    def __len__(self):
        len_ = 0
        for num in ['1', '2', '3', '4', '5']:
            with open('words_filter1_without_numbers' + num + '.pickle', 'rb') as f:
                len_ = len_ + len(pickle.load(f))
            
        return len_

    def __iter__(self):
        for num in ['1', '2', '3', '4', '5']:
            all_words = []
            with open('words_filter1_without_numbers_' + num + '.pickle', 'rb') as f:
                all_words = pickle.load(f)
            
            for url, doc, w in all_words:
                yield w

In [72]:
def filterWord(words,
               stopWords=set(stopwords.words("english")) |  set(stopwords.words("russian")),
               lemmerRus=Mystem(),
               lemmerEng=WordNetLemmatizer()):
    wordsFiltered = list()
    for w in words[2]:
        if len(w) <= 1 or len(w) > 20:
            continue
        
        w = ''.join(list(filter(lambda x: x.isalpha() or x.isdigit() or x==" " or x=="-", w)))
        if len(w) == 0:
            continue
        
        if w not in stopWords:
            w = lemmerEng.lemmatize(w)
            w = lemmerRus.lemmatize(w)
            for ww in w[:-1]:
                if len(ww) > 1:
                    wordsFiltered.append(ww)
    
    #print(wordsFiltered)
    return words[0], words[1], wordsFiltered

In [42]:
def deleteNumbers(words):
    wordsFiltered = list()
    for w in words[2]:
        
        w = ''.join(list(filter(lambda x: x.isalpha() or x==" " or x=="-", w)))
        if len(w) == 0:
            continue
        wordsFiltered.append(w)
    
    return words[0], words[1], wordsFiltered

In [43]:
%%time
for num in ['1', '2', '3', '4', '5']:
    with open('words_filter1_' + num + '.pickle', 'rb') as f:
        all_words = pickle.load(f)
    print(num + "\n")
    
    startTime = datetime.now()
    new_words = list(map(deleteNumbers, all_words))
    print(datetime.now() - startTime)
    
    with open('words_filter1_without_numbers_' + num + '.pickle', 'wb') as f:
        pickle.dump(new_words, f)
    new_words = []

1

0:00:30.273372
2

0:00:31.538136
3

0:00:32.034783
4

0:00:30.648446
5

0:00:31.844142
CPU times: user 3min 13s, sys: 11.4 s, total: 3min 25s
Wall time: 3min 32s


In [47]:
sentences = MySentences()

In [6]:
%%time
len(sentences)

CPU times: user 14.8 s, sys: 5.12 s, total: 19.9 s
Wall time: 20.4 s


74258

In [7]:
%%time
from gensim.summarization.bm25 import get_bm25_weights
bm25 = gensim.summarization.bm25.BM25(sentences)

CPU times: user 1min 3s, sys: 8.51 s, total: 1min 12s
Wall time: 1min 14s


In [8]:
#%%time
#bm25dict = gensim.corpora.Dictionary(sentences)

In [9]:
with open('bm25_4.pickle', 'wb') as f:
    pickle.dump(bm25, f)

In [45]:
with open('bm25dict.pickle', 'wb') as f:
    pickle.dump(bm25dict, f)

In [19]:
with open('bm25_4.pickle', 'rb') as f:
    bm25 = pickle.load(f)

In [21]:
len(bm25.doc_len)

74258

In [62]:
scores = []
for index in queries.index:
    print(index, end="\r")
    query = queries.iloc[index]['query']
    query_words = nltk.word_tokenize(query.lower())
    tmp = []
    for doc in range(len(bm25.doc_len)):
        tmp.append((index, doc, bm25.get_score(query_words, doc)))
    tmp = sorted(tmp, reverse=True, key=lambda pair: pair[2])[:95]
    scores.append(tmp)

In [20]:
subm_df = pd.read_csv("sample_submission.txt", sep=",")

In [49]:
## WITHOUT WORD2VEC
scores = []
for i in range(1, 1592):
    print(i, end="\r")
    query = queries.iloc[i-1]['query']
    query_words = nltk.word_tokenize(query.lower())
    
    tmp = []
    for ii in list(subm_df[subm_df['QueryId'] == i]['DocumentId']):
        doc = int(links[links['docId'] == ii]['index'])
        tmp.append((i, ii, bm25.get_score(query_words, doc)))
    tmp = sorted(tmp, reverse=True, key=lambda x: x[2])[:15]
    scores.append(tmp)

In [21]:
with open('w2v_dict1.pickle', 'rb') as f:
    w2v = pickle.load(f)

In [22]:
class Score:
    def __init__(self, doc):
        self.doc = doc
    
    def get_score(self, words):
        return bm25.get_score(list(words), self.doc)

In [23]:
w2v.most_similar("кот")

[('котенок', 0.8781003355979919),
 ('пес', 0.846825122833252),
 ('крыса', 0.8331175446510315),
 ('собачка', 0.830483078956604),
 ('кошка', 0.8209354877471924),
 ('котик', 0.8108201026916504),
 ('птичка', 0.8098441362380981),
 ('заяц', 0.8085563778877258),
 ('собака', 0.7970916032791138),
 ('кошечка', 0.7874215245246887)]

In [ ]:
# WITH WORD2VEC
scores = []
for i in range(1, 1592):
    print(i, end="\r")
    query = queries.iloc[i-1]['query']
    query_words = nltk.word_tokenize(query.lower())
    #print(query_words)
    
    count_ = 0
    for qw in query_words:
        try:
            count_ += 1
            tmp = w2v.most_similar(qw)
        except:
            pass
    
    if count_ <= 8:
        synonyms = {}
        for qw in query_words:
            
            tmp = []
            try:
                for ww, sc in w2v.most_similar(qw):
                    tmp.append(ww)
            except:
                pass
            synonyms[qw] = tmp

        query_words_new = []
        for q in query_words:
            if len(synonyms[q]) > 0:
                tl = synonyms[q][:5]
                tl.append(q)
                query_words_new.append(tl)
            else:
                tl = [q]
                query_words_new.append(tl)
        
        query_words_new = list(itertools.product(*query_words_new))
    else:
        query_words_new = [query_words]

    tmp_dict = {}
    for ii in list(subm_df[subm_df['QueryId'] == i]['DocumentId']):
        tmp_dict[ii] = -1

    for ii in list(subm_df[subm_df['QueryId'] == i]['DocumentId']):
        doc = int(links[links['docId'] == ii]['index'])
        sc = Score(doc)
        s = max(map(sc.get_score, query_words_new))
        tmp_dict[ii] = s

    tmp = []
    for ii in tmp_dict.keys():
        tmp.append((i, ii, tmp_dict[ii]))
    tmp = sorted(tmp, reverse=True, key=lambda x: x[2])[:15]

    scores.append(tmp)

In [ ]:
scores

In [ ]:
out = open("my_subm_10_TRY.txt", "w")
out.write("QueryId,DocumentId\n")
for iii, sc in enumerate(scores):
    # print(iii)
    for i in sc:
        out.write(str(i[0]) + "," + str(i[1]) + "\n")
out.close()

In [ ]:
out = open("my_subm_6.txt", "w")
dir_ = os.listdir("content/20190128/")
for j, score in enumerate(scores):
    jj = 0
    print(j, end="\r")
    for i in score:
        
        if jj >= 12:
            break
        
        ii = dir_[i[1]]
        
        try:
            iii = int(dff[dff['file'] == ii]['url_id'])
            out.write(str(i[0]+1) + "," + str(iii) + "\n")
            jj = jj + 1
        except ValueError:
            pass
out.close()

In [ ]:
w2v = gensim.models.Word2Vec(size=100, min_count=1, window=5)

In [ ]:
%%time
sentences = MySentences()
w2v.build_vocab(sentences)

In [ ]:
%%time
w2v.train(sentences, total_examples=w2v.corpus_count, epochs=1)

In [ ]:
#with open('w2v.pickle', 'wb') as f:
#    pickle.dump(w2v, f)

In [ ]:
with open('w2v_dict1.pickle', 'wb') as f:
        pickle.dump(w2v.wv, f)

In [ ]:
with open('w2v.pickle', 'rb') as f:
    w2v = pickle.load(f)

In [99]:
sum_ = 0
n = 0
for ii, sentence in enumerate(sentences):
    #print("1!!!!!!!!!!!!1")
    #print(type(i))
    #print(ii, end="\r")
    print(sentence)
    print()

['главн', 'страниц', 'решебник', 'ответ', 'кроссворд', 'поздравлен', 'послан', 'мен', 'поиск', 'слов', 'шаблон', 'инструкц', 'поиск', 'вопрос', 'вернут', 'страниц', 'списк', 'вопросовтонк', 'гибк', 'прут', 'упруг', 'плетк', 'ответ', 'смотр', 'ниж', 'вопрос', 'тонк', 'гибк', 'прут', 'упруг', 'плетк', 'сканворд', 'кроссворд.количесв', 'символ', 'буквответ', 'смотр', 'ниж', 'хлыстдруг', 'формулировк', 'слов', 'хлыст', 'картинкесрублен', 'дерев', 'вмест', 'вершин', 'очищен', 'сучьевупруг', 'плеткаещ', 'вопрос', 'легк', 'жил', 'постройк', 'некотор', 'стран', 'продукт', 'труд', 'имеющ', 'стоимост', 'распределя', 'пут', 'купли-продаж', 'взлетно-посадочн', 'устройств', 'самолет', 'выступа', 'из-под', 'кож', 'кост', 'online-tusa.c', 'может', 'остав', 'комментар', 'ваш', 'браузер', 'включ', 'поддержк', 'javascript']

['находун', 'возможн', 'иска', '``', 'прод', 'турник', 'актоб', 'запрос', 'куп', 'турник', 'актоб', 'беларус', 'найд', '24', 'объявлен', 'металлочерепиц', 'брест', 'металлическ', 'к

['нужн', 'клиент', 'добав', 'товар', 'услуг', 'каталог', 'satu.kz', 'зарегистрир', 'компан', 'спасиб', 'покупател', 'мобильн', 'каталог', 'товар', 'казахста', 'товары/регулятор', 'напряжен', 'генератор', 'найд', '336', 'напряжен', 'генератор', 'найд', '336', 'наименован', 'регулятор', 'напряжен', 'генератор', 'казахстан', 'генератор', 'электростанц', '181', 'электрогенератор', '141', 'комплект', 'генератор', '40', 'электрооборудован', 'автомобил', '94', 'генератор', 'альтернатор', 'комплект', '65', 'рел', 'датчик', 'автомобильн', '29', 'прибор', 'измерен', 'давлен', '26', 'регулятор', 'давлен', 'газ', '25', 'датчик', 'давлен', 'найд', '10', 'категор', 'дета', 'ходов', '13', 'рычаг', 'тяг', '13', 'автозапчаст', 'комплект', 'автозапчаст', 'комплект', 'общ', 'запчаст', 'дорожно-строительн', 'спецтехник', 'электрооборудован', 'грузов', 'автомобил', 'компонент', 'асутп', 'систем', 'автоматическ', 'управлен', 'кранов', 'оборудован', 'гусеничн', 'кран', 'запасн', 'част', 'комплект', 'кранов',

['найт', 'виде', 'русск', 'пытк', 'садо-маз', 'насил', 'бондаж', 'фемд', 'бдсм', 'порк', 'госпож', 'рабын', 'госпож', 'туалетн', 'рабын', 'русск', 'рабын', 'груб', 'бол', 'рабын', 'дрессировк', 'рабын', 'унижен', 'рабын', 'наказан', 'жен', 'рабын', 'эксцентричн', 'русск', 'бдсм', 'тюрьм', 'секс', 'рабын', 'латекс', 'русск', 'клизм', 'принужден', 'извращен', 'подчинен', 'раб', 'пирсинг', 'рабын', 'японск', 'школьниц', '18+', 'мамаш', 'парен', '18+', 'ге', 'раб', 'китаянк', 'сперм', 'рот', 'бесплатн', 'видео-порнух', 'тем', '``', 'раб', 'раб', 'главн', 'последн', 'популярн', 'продолжительн', '10', '11', '...', '20', '12:18', 'униз', 'бол', 'раб', 'унижен', 'рабын', 'порк', 'унижен', 'издевательств', 'бдсм', 'унижен', 'унижен', 'рабын', 'pain', 'униз', 'рабын', 'рабын', 'унижа', 'раб', 'униз', 'бол', 'год', 'назад', 'jizzbunk', '17:05', 'секс', 'цар', 'ана', 'рабын', 'любительск', 'рабын', 'секс', 'рабын', 'домашн', 'секс', 'рабын', 'секс', 'принцесс', 'год', 'назад', 'pornhub', '36:58', 

['автомоб', 'бизнес', 'дом', 'сем', 'домашн', 'уют', 'духовн', 'развит', 'ед', 'напитк', 'закон', 'здоров', 'интернет', 'искусств', 'развлечен', 'карьер', 'компьютер', 'красот', 'маркетинг', 'мод', 'новост', 'обществ', 'образован', 'отношен', 'публикац', 'написан', 'стат', 'путешеств', 'реклам', 'самосовершенствован', 'спорт', 'фитнес', 'технолог', 'финанс', 'хобб', 'fb', '.ru', 'жизн', 'экономик', 'наук', 'авт', 'отд', 'хай-тек', 'здоров', '10', 'продукт', 'котор', 'нужн', 'куша', 'худеющ', 'человек', 'завтрака', 'навред', 'организм', 'опасн', 'привычк', 'постоя', 'выдергива', 'волос', 'японск', 'метод', 'самоисцелен', 'минут', 'топ', 'продукт', 'останавлива', 'развит', 'диабет', 'полезн', 'употреблен', 'мед', 'кориц', 'кажд', 'ден', 'клещ', 'ресниц', 'поня', 'завел', 'паразит', 'признак', 'сердечн', 'приступ', 'котор', 'появля', 'месяц', 'признак', 'рак', 'печен', 'котор', 'мал', 'знает', 'гроз', 'шишк', 'ше', 'ух', 'определ', 'болезн', 'посмотр', 'лиц', 'как', 'процесс', 'происход',

['нужн', 'клиент', 'добав', 'товар', 'услуг', 'каталог', 'tiu.r', 'зарегистрир', 'компан', 'спасиб', 'покупател', 'войтизарегистрирова', 'каталог', 'товар', 'казан', 'товары/умывальник', 'хирургическ', 'найд', '228', 'хирургическ', 'найд', '228', 'наименован', 'умывальник', 'хирургическ', 'казан', 'сантехник', '183', 'кран', 'смесител', '169', 'умывальник', 'пьедестал', '10', 'кухон', 'мойк', 'показа', 'сифон', 'трап', 'сантехническ', 'медицинск', 'мебел', '24', 'мойк', 'хирургическ', '24', 'нейтральн', 'оборудован', 'horec', 'fast-food', 'ван', 'моечн', 'найд', 'категор', 'медицинск', 'оборудован', 'медицинск', 'ван', 'товар', 'люд', 'ограничен', 'возможн', 'товар', 'люд', 'ограничен', 'возможн', 'общ', 'санитарн', 'приспособлен', 'инвалид', 'мебел', 'ван', 'тумб', 'раковин', 'смотр', 'такжесмесител', 'смесител', 'умывальник', 'смесител', 'groh', 'смесител', 'локтев', 'смесител', 'раковин', 'смесител', 'ван', 'смесител', 'кран', 'смесител', 'кухн', 'смесител', 'умывальник', 'смесител'

...

['пп', 'фот', 'виде', 'прикол', 'главн', 'виде', 'фот', 'поиск', 'earthquak', 'resist', 'structur', 'steel', 'high', 'seismic', 'zone', 'виде', 'смотрет', 'онлайн', 'earthquak', 'resist', 'structur', 'steel', 'high', 'seismic', 'zone', 'информац', 'виде', 'продолжительн', 'виде', '00:21:19', 'количеств', 'лайк', 'качеств', 'виде', 'sd', 'размерн', 'виде', '2d', 'количеств', 'просмотр', '331', 'дат', 'публикац', 'интернет', '2015-05-22', 'загрузк', '...', 'cмотрет', 'виде', 'онлайн', 'скача', 'виде', 'earthquak', 'resist', 'structur', 'steel', 'high', 'seismic', 'zone', 'скача', 'виде', 'нажм', 'кнопк', 'получ', 'ссылк', 'сможет', 'выбра', 'необходим', 'формат', 'виде', 'mp4', 'webm', 'x-flv', '3gp', 'скача', 'получ', 'ссылк', 'описан', 'g.', 'krishnamurth', 'visit', 'facult', 'nirm', 'universit', 'sscs', 'ahmedabad', 'held', '22nd', 'apr', '2015', 'тег', 'structura', 'steel', 'construct', 'summitsteelsteel', 'constructionsteel', 'constructionsteel', 'construct', 'materialsteel', 'build

['главн', 'рубрик', 'программн', 'обеспечен', 'наш', 'программ', 'сервис', 'интернет', 'подарк', 'ценообразован', 'смет', 'письм', 'минстро', 'пожарн', 'безопасн', 'источник', 'питан', 'проектирован', 'охра', 'сигнализац', 'охра', 'деятельн', 'монтаж', 'пожарн', 'сигнализац', 'энциклопед', 'безопасн', 'юмор', 'безопасн', 'рул', 'безопасн', 'бизнес', 'полезн', 'информац', 'личн', 'безопасн', 'наладк', 'видеонаблюден', 'оповещен', 'эвакуац', 'техническ', 'обслуживан', 'контрол', 'доступ', 'пожаротушен', 'документац', 'прайс', 'карт', 'сайт', 'подарк', 'интернет-магазин', 'пожарн', 'сигнализац', 'расстановк', 'пожарн', 'извещател', '10.02.2012', 'отечествен', 'зарубежн', 'норм', 'прошедш', 'год', 'мног', 'нормативн', 'положен', 'определя', 'размещен', 'пожарн', 'извещател', 'успел', 'поменя', 'раз', 'смен', 'нпб', '88-2001*', 'установк', 'пожаротушен', 'сигнализац', 'норм', 'прав', 'проектирован', 'ноябр', '2008', 'г.', 'вышел', 'нов', 'свод', 'прав', 'сп', '5.13130.2009', 'систем', 'прот

['английск', 'онлайн', 'индивидуальн', 'подход', 'toggl', 'navig', 'проект', 'руководител', 'проект', 'эксперт', 'курс', 'тренинг', 'расписан', 'мероприят', 'программ', 'вебинар', 'каталог', 'бесплатн', 'стат', 'остав', 'отз', 'english', 'cup', 'te', 'стат', 'расширя', 'словарн', 'запас', 'врем', 'по-английск', 'врем', 'по-английск', 'автор', 'людм', 'хабиров', '2016-05-01', 'комментар', 'tick-tock', 'say', 'clock', 'tim', 'pla', 'tim', 'walk', 'такж', 'tim', 'work', 'stud', 'сегодн', 'разбер', 'правильн', 'озвуч', 'врем', 'английск', 'нужн', 'сказа', 'сколько-т', 'час', 'ровн', 'применя', 'слов', 'clock', 'six', 'clock', 'шест', 'час', 'мож', 'добав', 'слов', 'sharp', 'exactl', 'усилен', 'обрат', 'вниман', 'разн', 'положен', 'эт', 'слов', 'предложен', 'six', 'clock', 'sharp', 'ровн', 'шест', 'час', 'exactl', 'six', 'clock', 'ровн', 'шест', 'час', 'наоборот', 'нужн', 'подчеркнут', 'увер', 'точност', 'времен', 'использ', 'слов', 'around', 'ten', 'окол', '10', 'час', 'around', 'ten', 'пр

['skip', 'content', 'главн', 'карт', 'сайт', 'контакт', 'медицин', 'народ', 'народн', 'рецепт', 'применен', 'средств', 'народн', 'медицин', 'использован', 'народн', 'средств', 'лечен', 'распространен', 'заболеван', 'правильн', 'применя', 'народн', 'рецепт', 'men', 'аденом', 'алкоголизм', 'бесплод', 'бронх', 'молочниц', 'потенц', 'простат', 'простуд', 'пневмон', 'перхот', 'эроз', 'нов', 'эндопротезирован', 'тазобедрен', 'сустав', 'москв', 'сут', 'операц', 'каков', 'альтернатив', 'нов', 'биполярн', 'расстройств', 'личностинов', 'выбира', 'шампун', 'прот', 'выпаден', 'волоснов', 'как', 'способ', 'проверя', 'бесплодиенов', 'лечен', 'кардиологическ', 'заболеван', 'проверен', 'клиник', 'народн', 'рецептынов', 'народн', 'метод', 'снижен', 'уровн', 'холестерин', 'правильн', 'питан', 'мен', 'деньнов', 'лечен', 'гидроцефал', 'израиленов', 'ингаляц', 'обструктивн', 'бронх', 'hom', 'потлив', 'потлив', 'врем', 'сна', 'потлив', 'врем', 'сна', 'swayoleg', 'люд', 'страда', 'ночн', 'выделен', 'пот', 'э

['знаменитостиид', 'домамод', 'стильсовет', 'поиск', 'lad', 'wom', 'smart', 'magazin', 'знаменитостиид', 'домамод', 'стильсовет', 'фот', 'ид', 'красив', 'клумб', 'сво', 'рук', 'подручн', 'сдела', 'красив', 'клумб', 'сво', 'рук', 'очен', 'прост', 'понадоб', 'небольш', 'навык', 'немн', 'терпен', 'подручн', 'материал', 'выбра', 'подходя', 'вариант', 'клумб', 'цветник', 'почерпнут', 'ид', 'клумб', 'сво', 'рук', 'поможет', 'наш', 'стат', 'расскаж', 'покаж', 'прост', 'материал', 'получа', 'сооруд', 'оригинальн', 'красив', 'клумб', 'собствен', 'рук', 'наверняк', 'дом', 'дач', 'затеря', 'пар', 'стар', 'покрышек', 'пластиков', 'стекля', 'бутылк', 'недавн', 'что—т', 'стро', 'такж', 'должн', 'оста', 'строительн', 'материал', 'кирпич', 'камн', 'дерево.возможн', 'сара', 'хран', 'стар', 'мебел', 'котор', 'давн', 'польз', 'жалк', 'выкинут', 'так', 'подручн', 'материал', 'сооруд', 'красив', 'оригинальн', 'клумб', 'сво', 'рук', 'украс', 'ваш', 'участок', 'дом', 'прекрасн', 'цветник', 'котор', 'радова',

In [39]:
doc_indexes = {}
for index in urls.index:
    doc_indexes[urls.iloc[index]['url']] = int(urls.iloc[index]['url_id'])